In [1]:
import h5py
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import shap
import lightgbm as lgb 
import optuna
from optuna.samplers import TPESampler
from optuna.integration import LightGBMPruningCallback
from optuna.pruners import MedianPruner
import xgboost

In [14]:
# read data
def load_data(name):
    with h5py.File(f'{name}.h5', 'r') as f:
        filename = name.split('/')[-1]
        return pandas.DataFrame(f[filename][:], dtype=np.float64)

train = load_data('train')
test  = load_data('test')

print (f'Shape of training data set: {train.shape}')
print (f'Shape of test data set: {test.shape}')

all_variables = ['actualInteractionsPerCrossing', 'averageInteractionsPerCrossing', 'correctedActualMu', 'correctedAverageMu', 'correctedScaledActualMu', 'correctedScaledAverageMu', 'NvtxReco', 'p_nTracks', 'p_pt_track', 'p_eta', 'p_phi', 'p_charge', 'p_qOverP', 'p_z0', 'p_d0', 'p_sigmad0', 'p_d0Sig', 'p_EptRatio', 'p_dPOverP', 'p_z0theta', 'p_etaCluster', 'p_phiCluster', 'p_eCluster', 'p_rawEtaCluster', 'p_rawPhiCluster', 'p_rawECluster', 'p_eClusterLr0', 'p_eClusterLr1', 'p_eClusterLr2', 'p_eClusterLr3', 'p_etaClusterLr1', 'p_etaClusterLr2', 'p_phiClusterLr2', 'p_eAccCluster', 'p_f0Cluster', 'p_etaCalo', 'p_phiCalo', 'p_eTileGap3Cluster', 'p_cellIndexCluster', 'p_phiModCalo', 'p_etaModCalo', 'p_dPhiTH3', 'p_R12', 'p_fTG3', 'p_weta2', 'p_Reta', 'p_Rphi', 'p_Eratio', 'p_f1', 'p_f3', 'p_Rhad', 'p_Rhad1', 'p_deltaEta1', 'p_deltaPhiRescaled2', 'p_TRTPID', 'p_TRTTrackOccupancy', 'p_numberOfInnermostPixelHits', 'p_numberOfPixelHits', 'p_numberOfSCTHits', 'p_numberOfTRTHits', 'p_numberOfTRTXenonHits', 'p_chi2', 'p_ndof', 'p_SharedMuonTrack', 'p_E7x7_Lr2', 'p_E7x7_Lr3', 'p_E_Lr0_HiG', 'p_E_Lr0_LowG', 'p_E_Lr0_MedG', 'p_E_Lr1_HiG', 'p_E_Lr1_LowG', 'p_E_Lr1_MedG', 'p_E_Lr2_HiG', 'p_E_Lr2_LowG', 'p_E_Lr2_MedG', 'p_E_Lr3_HiG', 'p_E_Lr3_LowG', 'p_E_Lr3_MedG', 'p_ambiguityType', 'p_asy1', 'p_author', 'p_barys1', 'p_core57cellsEnergyCorrection', 'p_deltaEta0', 'p_deltaEta2', 'p_deltaEta3', 'p_deltaPhi0', 'p_deltaPhi1', 'p_deltaPhi2', 'p_deltaPhi3', 'p_deltaPhiFromLastMeasurement', 'p_deltaPhiRescaled0', 'p_deltaPhiRescaled1', 'p_deltaPhiRescaled3', 'p_e1152', 'p_e132', 'p_e235', 'p_e255', 'p_e2ts1', 'p_ecore', 'p_emins1', 'p_etconeCorrBitset', 'p_ethad', 'p_ethad1', 'p_f1core', 'p_f3core', 'p_maxEcell_energy', 'p_maxEcell_gain', 'p_maxEcell_time', 'p_maxEcell_x', 'p_maxEcell_y', 'p_maxEcell_z', 'p_nCells_Lr0_HiG', 'p_nCells_Lr0_LowG', 'p_nCells_Lr0_MedG', 'p_nCells_Lr1_HiG', 'p_nCells_Lr1_LowG', 'p_nCells_Lr1_MedG', 'p_nCells_Lr2_HiG', 'p_nCells_Lr2_LowG', 'p_nCells_Lr2_MedG', 'p_nCells_Lr3_HiG', 'p_nCells_Lr3_LowG', 'p_nCells_Lr3_MedG', 'p_pos', 'p_pos7', 'p_poscs1', 'p_poscs2', 'p_ptconeCorrBitset', 'p_ptconecoreTrackPtrCorrection', 'p_r33over37allcalo', 'p_topoetconeCorrBitset', 'p_topoetconecoreConeEnergyCorrection', 'p_topoetconecoreConeSCEnergyCorrection', 'p_weta1', 'p_widths1', 'p_widths2', 'p_wtots1', 'p_e233', 'p_e237', 'p_e277', 'p_e2tsts1', 'p_ehad1', 'p_emaxs1', 'p_fracs1', 'p_DeltaE', 'p_E3x5_Lr0', 'p_E3x5_Lr1', 'p_E3x5_Lr2', 'p_E3x5_Lr3', 'p_E5x7_Lr0', 'p_E5x7_Lr1', 'p_E5x7_Lr2', 'p_E5x7_Lr3', 'p_E7x11_Lr0', 'p_E7x11_Lr1', 'p_E7x11_Lr2', 'p_E7x11_Lr3', 'p_E7x7_Lr0', 'p_E7x7_Lr1' ]

X = train[all_variables]
y = train['Truth']

print (f'Shape of X: {X.shape}')
print (f'Shape of y: {y.shape}')

print(sum(y==0))
print(sum(y==1))

Shape of training data set: (162500, 166)
Shape of test data set: (160651, 164)
Shape of X: (162500, 160)
Shape of y: (162500,)
41005
121495


In [3]:
# split the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
# decision tree
def decision_tree(X_train, X_test, y_train, y_test):


    clf = DecisionTreeClassifier()

    model = clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return model, y_pred, accuracy


In [5]:
# shap value xgbooster

def shap_xgbooster():
    model = xgboost.XGBRegressor().fit(X, y)
    explainer = shap.Explainer(model)
    shap_values = explainer(X)

    feature_names = shap_values.feature_names
    shap_df = pd.DataFrame(shap_values.values, columns=feature_names)
    vals = np.abs(shap_df.values).mean(0)
    shap_importance = pd.DataFrame(list(zip(feature_names, vals)), columns=['col_name', 'feature_importance_vals'])
    shap_importance.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)
    shap.plots.bar(shap_values)

    return shap_importance.head(20)

shape_variables = ['p_Rhad', 'p_Rphi', 'p_Reta', 'p_sigmad0', 'p_deltaEta1', 'p_ptconecoreTrackPtrCorrection', 'p_deltaPhiRescaled2', 'p_d0', 'p_numberOfInnermostPixelHits', 'p_ambiguityType',
                    'p_rawPhiCluster','p_phiCalo', 'p_ethad', 'p_EptRatio', 'p_Rhad1', 'p_E7x11_Lr3', 'p_ehad1', 'p_Eratio', 'p_deltaPhi2', 'p_nTracks']

shap_variables = ['p_Rhad','p_Reta','p_deltaEta1', 'p_sigmad0',
	'p_Rphi',	
	'p_ambiguityType',	
	'p_ethad',
	'p_numberOfInnermostPixelHits',
	'p_deltaPhiRescaled2',
	'p_ptconecoreTrackPtrCorrection',
	'p_d0',
	'p_Rhad1',
	'p_d0Sig',
	'p_nTracks',
	'p_deltaPhiFromLastMeasurement',
	'p_E7x11_Lr3',
	'p_deltaPhi2',
	'p_numberOfPixelHits',	
	'p_EptRatio',	
	'p_dPOverP']


In [6]:
# feature ranking lgb

def feature_importance_lgb():
    gbm = lgb.LGBMRegressor()
    gbm.fit(X_train, y_train)
    gbm.booster_.feature_importance()

    feature_imp_ = pd.DataFrame({'cols':X_train.columns, 'feature_imp':gbm.feature_importances_})
    feature_imp_.loc[feature_imp_.feature_imp > 0].sort_values(by=['feature_imp'], ascending=False)

    return feature_imp_.loc[feature_imp_.feature_imp > 0].sort_values(by=['feature_imp'], ascending=False).head(20)

feature_importance_lgb()
    

feature_var_lgb = ['p_sigmad0', 'p_deltaPhiRescaled2', 'p_deltaEta1', 'p_Reta', 'p_d0', 'p_ptconecoreTrackPtrCorrection',
'p_d0Sig', 'p_Rhad', 'p_Rphi', 'p_deltaPhiFromLastMeasurement', 'p_deltaPhi2', 'p_EptRatio', 'p_dPOverP', 'p_numberOfSCTHits',
 'p_numberOfPixelHits', 'p_pt_track', 'p_nTracks', 'p_ethad', 'p_qOverP','p_ambiguityType']



In [7]:
import optuna.integration.lightgbm as oplgb
import optuna
from sklearn.model_selection import RepeatedKFold

X_lgb = train[feature_var_lgb]
y_lgb = train['Truth']

X_lgb_train, X_lgb_test, y_lgb_train, y_lgb_test = train_test_split(X_lgb, y_lgb, test_size=0.2, random_state=42)

def objective_lgb():
    
    train_data = lgb.Dataset(X_lgb_train, label=y_lgb_train)
    valid_data = lgb.Dataset(X_lgb_test, label=y_lgb_test)
    
    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
    params = {
            'objective': 'regression',
            'metric': 'binary_logloss',
            'boosting_type': 'dart',
            'max_depth': 10,
            'learning_rate': 0.2759844445088989,
            'feature_fraction': 0.8,
            'num_leaves': 360,
            'bagging_freq': 1,
            'bagging_fraction': 0.8,
            'reg_alpha': 8.200000000000001,
            'reg_lambda': 103,
            'verbose':-1,
            'force_col_wise': True
    }

    study_tuner = optuna.create_study(
    direction="minimize")

    tuner = oplgb.LightGBMTunerCV(params, train_data, study=study_tuner, num_boost_round=100, folds=rkf, 
                                early_stopping_rounds=200, seed=42)

    tuner.run()

    return tuner.best_params


objective_lgb()

# {'objective': 'regression',
#  'metric': 'binary_logloss',
#  'boosting_type': 'dart',
#  'max_depth': 10,
#  'learning_rate': 0.2759844445088989,
#  'feature_fraction': 0.9520000000000001,
#  'num_leaves': 360,
#  'bagging_freq': 1,
#  'bagging_fraction': 0.9993807771263824,
#  'verbose': -1,
#  'force_col_wise': True,
#  'feature_pre_filter': False,
#  'lambda_l1': 8.200000000000001,
#  'lambda_l2': 103,
#  'min_child_samples': 20}



[I 2022-05-20 13:37:26,152] A new study created in memory with name: no-name-d15c7029-4295-417c-afbc-4271de154554
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[1]	cv_agg's binary_logloss: 0.414061 + 0.00242977


Early stopping is not available in dart mode


[2]	cv_agg's binary_logloss: 0.331188 + 0.00238197
[3]	cv_agg's binary_logloss: 0.278093 + 0.00248272
[4]	cv_agg's binary_logloss: 0.242564 + 0.00260245
[5]	cv_agg's binary_logloss: 0.217981 + 0.00275328
[6]	cv_agg's binary_logloss: 0.200521 + 0.00285777
[7]	cv_agg's binary_logloss: 0.188057 + 0.00315571
[8]	cv_agg's binary_logloss: 0.201277 + 0.00283582
[9]	cv_agg's binary_logloss: 0.188477 + 0.00315403
[10]	cv_agg's binary_logloss: 0.17928 + 0.00327037
[11]	cv_agg's binary_logloss: 0.172744 + 0.00346688
[12]	cv_agg's binary_logloss: 0.177258 + 0.00337923
[13]	cv_agg's binary_logloss: 0.171271 + 0.00342443
[14]	cv_agg's binary_logloss: 0.167107 + 0.00366153
[15]	cv_agg's binary_logloss: 0.164342 + 0.00400397
[16]	cv_agg's binary_logloss: 0.163014 + 0.00448844
[17]	cv_agg's binary_logloss: 0.162615 + 0.00511034
[18]	cv_agg's binary_logloss: 0.16364 + 0.00664395
[19]	cv_agg's binary_logloss: 0.165487 + 0.00725046
[20]	cv_agg's binary_logloss: 0.168174 + 0.00808852
[21]	cv_agg's binary_l

feature_fraction, val_score: 0.158008:  14%|#4        | 1/7 [01:24<08:24, 84.07s/it]'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


...

[1]	cv_agg's binary_logloss: 0.413 + 0.00246201


Early stopping is not available in dart mode


[2]	cv_agg's binary_logloss: 0.329831 + 0.00231621
[3]	cv_agg's binary_logloss: 0.276444 + 0.00239898
[4]	cv_agg's binary_logloss: 0.241034 + 0.00258902
[5]	cv_agg's binary_logloss: 0.216595 + 0.00272511
[6]	cv_agg's binary_logloss: 0.199016 + 0.00288878
[7]	cv_agg's binary_logloss: 0.186613 + 0.00302495
[8]	cv_agg's binary_logloss: 0.200129 + 0.00284244
[9]	cv_agg's binary_logloss: 0.187147 + 0.0029291
[10]	cv_agg's binary_logloss: 0.178086 + 0.00321161
[11]	cv_agg's binary_logloss: 0.171645 + 0.00330954
[12]	cv_agg's binary_logloss: 0.176181 + 0.00316377
[13]	cv_agg's binary_logloss: 0.170214 + 0.00340151
[14]	cv_agg's binary_logloss: 0.165836 + 0.00369875
[15]	cv_agg's binary_logloss: 0.162962 + 0.00398296
[16]	cv_agg's binary_logloss: 0.160951 + 0.00417142
[17]	cv_agg's binary_logloss: 0.160665 + 0.0046288
[18]	cv_agg's binary_logloss: 0.161101 + 0.00529865
[19]	cv_agg's binary_logloss: 0.162523 + 0.00587703
[20]	cv_agg's binary_logloss: 0.16498 + 0.00629634
[21]	cv_agg's binary_lo

min_data_in_leaf, val_score: 0.156539: 100%|##########| 5/5 [06:58<00:00, 83.65s/it]


{'objective': 'regression',
 'metric': 'binary_logloss',
 'boosting_type': 'dart',
 'max_depth': 10,
 'learning_rate': 0.2759844445088989,
 'feature_fraction': 0.9520000000000001,
 'num_leaves': 360,
 'bagging_freq': 1,
 'bagging_fraction': 0.9993807771263824,
 'verbose': -1,
 'force_col_wise': True,
 'feature_pre_filter': False,
 'lambda_l1': 8.200000000000001,
 'lambda_l2': 103,
 'min_child_samples': 20}

In [8]:
def optimized_lgb(X_train, X_test, y_train, y_test):
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test)
    params = {'objective': 'regression',
            'metric': 'binary_logloss',
            'boosting_type': 'dart',
            'max_depth': 10,
            'learning_rate': 0.2759844445088989,
            'feature_fraction': 0.8,
            'num_leaves': 360,
            'bagging_freq': 1,
            'bagging_fraction': 0.8,
            'reg_alpha': 8.200000000000001,
            'reg_lambda': 103,
            'verbose':-1,
            'force_col_wise': True}

    lgb_clf = lgb.train(params, train_set=train_data, num_boost_round=1000)
    y_pred = np.around(lgb_clf.predict(X.values))
    acc = accuracy_score(y_pred, y)
    print(f"Train accuracy: {acc*100.0:.2f}%")

    return lgb_clf.predict(X.values)

In [9]:
shape_variables = ['p_Rhad', 'p_Rphi', 'p_Reta', 'p_sigmad0', 'p_deltaEta1', 'p_ptconecoreTrackPtrCorrection', 'p_deltaPhiRescaled2', 'p_d0', 'p_numberOfInnermostPixelHits', 'p_ambiguityType',
                    'p_rawPhiCluster','p_phiCalo', 'p_ethad', 'p_EptRatio', 'p_Rhad1', 'p_E7x11_Lr3', 'p_ehad1', 'p_Eratio', 'p_deltaPhi2', 'p_nTracks']

# split data with shap
X = train[shape_variables]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [10]:
# print(np.shape(decision_tree(X_train, X_test, y_train, y_test)))

In [11]:
# optimize hyperparameters of lgb
# train_data = lgb.Dataset(X_train, label=y_train)
# valid_data = lgb.Dataset(X_test, label=y_test)

# def objective_lgb(trial):
    
#     train_data = lgb.Dataset(X_train, label=y_train)
#     valid_data = lgb.Dataset(X_test, label=y_test)

#     boosting_types = ["gbdt", "rf", "dart"]
#     boosting_type = trial.suggest_categorical("boosting_type", boosting_types)

#     params = {
#         'objective': 'regression',
#         'boosting_type': trial.suggest_categorical("boosting_type", ["gbdt", "rf", "dart"]),
#         'max_depth': trial.suggest_int('max_depth', 2, 12),
#         'metric': {'l2', 'auc'},
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.2, 0.95, step=0.1),
#         'num_leaves': trial.suggest_int('num_leaves', 20, 3000, step=20),
#         'bagging_freq': trial.suggest_categorical('bagging_freq', [1]),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.2, 0.95, step=0.1),
#         'reg_alpha': trial.suggest_float("reg_alpha", 0, 100, step=0.1),
#         'reg_lambda': trial.suggest_int("reg_lambda", 0, 1000, step=1),
#         'verbosity': -1,
#     }

#     N_iterations_max = 10000
#     early_stopping_rounds = 50

#     if boosting_type == "dart":
#         N_iterations_max = 100
#         early_stopping_rounds = None

#     cv_res = lgb.cv(
#         params,
#         train_data,
#         num_boost_round=N_iterations_max,
#         early_stopping_rounds=early_stopping_rounds,
#         verbose_eval=False,
#         seed=42,
#         callbacks=[LightGBMPruningCallback(trial, "binary_logloss")],
#     )

#     num_boost_round = len(cv_res["auc-mean"])
#     trial.set_user_attr("num_boost_round", num_boost_round)

#     return cv_res["auc-mean"][-1]



# study = optuna.create_study(
#     direction="maximize",
#     sampler=TPESampler(seed=42),
#     pruner=MedianPruner(n_warmup_steps=50),
# )

# study.optimize(objective_lgb, n_trials=100, show_progress_bar=True)

# study.best_trial.params

In [12]:
# from sklearn.metrics import roc_curve
# from sklearn.metrics import auc
# from sklearn.metrics import confusion_matrix

# fpr, tpr, _ = roc_curve(y_test1, y_score) 
# auc_score = auc(fpr,tpr)   
# auc_score

In [13]:
# optimized lgb

# def optimized_lgb(X_train, X_test, y_train, y_test):
#     train_data = lgb.Dataset(X_train, label=y_train)
#     test_data = lgb.Dataset(X_test, label=y_test)
#     params = {'objective': 'regression',
#             'metric': 'binary_logloss',
#             'boosting_type': 'dart',
#             'max_depth': 10,
#             'learning_rate': 0.2759844445088989,
#             'feature_fraction': 0.8,
#             'num_leaves': 360,
#             'bagging_freq': 1,
#             'bagging_fraction': 0.8,
#             'reg_alpha': 8.200000000000001,
#             'reg_lambda': 103,
#             'verbose':-1,
#             'force_col_wise': True}

#     lgb_clf = lgb.train(params, train_set=train_data, num_boost_round=1000)
#     y_pred = np.around(lgb_clf.predict(X.values))
#     acc = accuracy_score(y_pred, y)
#     print(f"Train accuracy: {acc*100.0:.2f}%")

#     return lgb_clf.predict(X.values), acc
    

{'boosting_type': 'dart',
 'max_depth': 10,
 'learning_rate': 0.2759844445088989,
 'feature_fraction': 0.8,
 'num_leaves': 360,
 'bagging_freq': 1,
 'bagging_fraction': 0.8,
 'reg_alpha': 8.200000000000001,
 'reg_lambda': 103}